In [ ]:
import os
os.listdir()


['.config', 'sample_data']

In [ ]:
import json
import pandas as pd

# Load Kp JSON
with open("/KP_DATA.json", "r") as f:
    kp_data = json.load(f)

kp_df = pd.DataFrame({
    "datetime": kp_data["datetime"],
    "kp": kp_data["Kp"]
})

kp_df["datetime"] = pd.to_datetime(kp_df["datetime"], utc=True)
kp_df = kp_df.sort_values("datetime")

kp_df.head(), kp_df.tail()


(                   datetime     kp
 0 2018-01-01 00:00:00+00:00  3.333
 1 2018-01-01 03:00:00+00:00  3.667
 2 2018-01-01 06:00:00+00:00  2.333
 3 2018-01-01 09:00:00+00:00  2.333
 4 2018-01-01 12:00:00+00:00  2.667,
                        datetime     kp
 23195 2025-12-09 09:00:00+00:00  0.667
 23196 2025-12-09 12:00:00+00:00  1.333
 23197 2025-12-09 15:00:00+00:00  1.333
 23198 2025-12-09 18:00:00+00:00  1.333
 23199 2025-12-09 21:00:00+00:00  1.000)

In [ ]:
import pandas as pd

files = [
    "/OMNIWeb 18-20.txt",
    "/OMNIWeb 20-22.txt",
    "/OMNIWeb 23-24.txt"
]

col_names = ["YEAR", "DOY", "HR", "Bt", "Bz", "Np", "V"]

omni_dfs = []

for f in files:
    df = pd.read_csv(
        f,
        sep=r"\s+",              # whitespace separator (future-proof)
        names=col_names,
        engine="python",         # more tolerant parser
        comment="#",             # skip comment lines
        on_bad_lines="skip"      # 🔥 skip malformed rows safely
    )
    omni_dfs.append(df)

omni_df = pd.concat(omni_dfs, ignore_index=True)

omni_df.head(), omni_df.tail()


(       YEAR          DOY       HR        Bt      Bz   Np   V
 0   OMNIWeb         Plus  Browser   Results    None  NaN NaN
 1  Selected  parameters:     None      None    None  NaN NaN
 2         1       Scalar       B,        nT    None  NaN NaN
 3         2          BZ,       nT     (GSM)    None  NaN NaN
 4         3           SW   Proton  Density,  N/cm^3  NaN NaN,
        YEAR  DOY HR    Bt    Bz   Np      V
 61857  2024  339  2  10.1   2.8  3.1  459.0
 61858  2024  339  3  10.2   4.0  3.0  459.0
 61859  2024  339  4   9.7   1.5  3.4  455.0
 61860  2024  339  5   9.4   3.3  3.6  478.0
 61861  2024  339  6   9.3  None  NaN    NaN)

In [ ]:
# Force numeric conversion (invalid values become NaN)
for col in ["YEAR", "DOY", "HR", "Bt", "Bz", "Np", "V"]:
    omni_df[col] = pd.to_numeric(omni_df[col], errors="coerce")

# Drop rows where time info is missing
omni_df = omni_df.dropna(subset=["YEAR", "DOY", "HR"])

# Convert time columns to int safely
omni_df["YEAR"] = omni_df["YEAR"].astype(int)
omni_df["DOY"] = omni_df["DOY"].astype(int)
omni_df["HR"] = omni_df["HR"].astype(int)

omni_df.head()


,YEAR,DOY,HR,Bt,Bz,Np,V
7,2018,1,0,8.2,-5.0,13.8,381.0
8,2018,1,1,10.8,-6.1,16.8,406.0
9,2018,1,2,10.0,-0.3,17.7,392.0
10,2018,1,3,10.9,-2.5,22.4,396.0
11,2018,1,4,11.0,2.4,19.0,416.0


In [ ]:
omni_df["datetime"] = pd.to_datetime(
    omni_df["YEAR"].astype(str) +
    omni_df["DOY"].astype(str).str.zfill(3) +
    omni_df["HR"].astype(str).str.zfill(2),
    format="%Y%j%H",
    utc=True
)

# Keep only required columns
omni_df = omni_df[["datetime", "V", "Np", "Bz", "Bt"]]

# Sort
omni_df = omni_df.sort_values("datetime")

omni_df.head(), omni_df.tail()


(                    datetime      V    Np   Bz    Bt
 7  2018-01-01 00:00:00+00:00  381.0  13.8 -5.0   8.2
 8  2018-01-01 01:00:00+00:00  406.0  16.8 -6.1  10.8
 9  2018-01-01 02:00:00+00:00  392.0  17.7 -0.3  10.0
 10 2018-01-01 03:00:00+00:00  396.0  22.4 -2.5  10.9
 11 2018-01-01 04:00:00+00:00  416.0  19.0  2.4  11.0,
                        datetime      V   Np   Bz    Bt
 61857 2024-12-04 02:00:00+00:00  459.0  3.1  2.8  10.1
 61858 2024-12-04 03:00:00+00:00  459.0  3.0  4.0  10.2
 61859 2024-12-04 04:00:00+00:00  455.0  3.4  1.5   9.7
 61860 2024-12-04 05:00:00+00:00  478.0  3.6  3.3   9.4
 61861 2024-12-04 06:00:00+00:00    NaN  NaN  NaN   9.3)

In [ ]:
omni_df.columns


Index(['datetime', 'V', 'Np', 'Bz', 'Bt'], dtype='object')

In [ ]:
omni_3h = (
    omni_df
    .set_index("datetime")
    .resample("3H")
    .mean()
    .reset_index()
)

omni_3h.head(), omni_3h.tail()


/tmp/ipython-input-4081892315.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("3H")


(                   datetime           V         Np        Bz         Bt
 0 2018-01-01 00:00:00+00:00  393.000000  16.100000 -3.800000   9.666667
 1 2018-01-01 03:00:00+00:00  417.666667  17.166667 -0.866667  10.066667
 2 2018-01-01 06:00:00+00:00  437.000000   8.700000 -0.466667   6.900000
 3 2018-01-01 09:00:00+00:00  433.000000   7.033333 -0.833333   6.966667
 4 2018-01-01 12:00:00+00:00  443.333333   5.266667 -1.533333   6.566667,
                        datetime           V        Np        Bz         Bt
 20230 2024-12-03 18:00:00+00:00  451.666667  3.833333  4.366667   9.900000
 20231 2024-12-03 21:00:00+00:00  457.000000  3.500000  1.900000   9.833333
 20232 2024-12-04 00:00:00+00:00  463.000000  3.400000  2.200000  10.133333
 20233 2024-12-04 03:00:00+00:00  464.000000  3.333333  2.933333   9.766667
 20234 2024-12-04 06:00:00+00:00         NaN       NaN       NaN   9.300000)

In [ ]:
final_df = pd.merge(
    omni_3h,
    kp_df,
    on="datetime",
    how="inner"
)

final_df.head(), final_df.tail()


(                   datetime           V         Np        Bz         Bt     kp
 0 2018-01-01 00:00:00+00:00  393.000000  16.100000 -3.800000   9.666667  3.333
 1 2018-01-01 03:00:00+00:00  417.666667  17.166667 -0.866667  10.066667  3.667
 2 2018-01-01 06:00:00+00:00  437.000000   8.700000 -0.466667   6.900000  2.333
 3 2018-01-01 09:00:00+00:00  433.000000   7.033333 -0.833333   6.966667  2.333
 4 2018-01-01 12:00:00+00:00  443.333333   5.266667 -1.533333   6.566667  2.667,
                        datetime           V        Np        Bz         Bt  \
 20230 2024-12-03 18:00:00+00:00  451.666667  3.833333  4.366667   9.900000   
 20231 2024-12-03 21:00:00+00:00  457.000000  3.500000  1.900000   9.833333   
 20232 2024-12-04 00:00:00+00:00  463.000000  3.400000  2.200000  10.133333   
 20233 2024-12-04 03:00:00+00:00  464.000000  3.333333  2.933333   9.766667   
 20234 2024-12-04 06:00:00+00:00         NaN       NaN       NaN   9.300000   
 
           kp  
 20230  3.333  
 20231  1.6

In [ ]:
final_df = final_df.dropna()

final_df.info()
final_df.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 20234 entries, 0 to 20233
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  20234 non-null  datetime64[ns, UTC]
 1   V         20234 non-null  float64            
 2   Np        20234 non-null  float64            
 3   Bz        20234 non-null  float64            
 4   Bt        20234 non-null  float64            
 5   kp        20234 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(5)
memory usage: 1.1 MB


,V,Np,Bz,Bt,kp
count,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000
mean,510.320553,18.343722,4.773564,10.338356,1.598801
std,921.197724,100.672855,67.019189,66.659235,1.219733
min,260.000000,0.133333,-31.833333,0.900000,0.000000
25%,345.666667,3.366667,-1.300000,3.733333,0.667000
50%,391.000000,5.200000,-0.133333,4.833333,1.333000
75%,458.666667,8.233333,1.066667,6.500000,2.333000
max,9999.000000,999.900000,999.900000,999.900000,9.000000


In [ ]:
# Remove OMNI bad-data flags
final_df = final_df[
    (final_df["V"] < 2000) &
    (final_df["Np"] < 100) &
    (final_df["Bt"] < 100) &
    (final_df["Bz"].abs() < 100)
]

# Recheck stats
final_df.describe()


,V,Np,Bz,Bt,kp
count,19890.000000,19890.000000,19890.000000,19890.000000,19890.000000
mean,408.642705,6.428185,-0.121224,5.468278,1.596571
std,84.219256,4.703818,2.539600,2.798590,1.218565
min,260.000000,0.133333,-31.833333,0.900000,0.000000
25%,345.333333,3.366667,-1.333333,3.733333,0.667000
50%,390.000000,5.133333,-0.133333,4.800000,1.333000
75%,455.000000,8.033333,1.033333,6.466667,2.333000
max,968.000000,70.666667,26.900000,56.666667,9.000000


In [ ]:
final_df.to_csv("geomagnetic_storm_ml_dataset_clean.csv", index=False)


In [ ]:

df = pd.read_csv("geomagnetic_storm_ml_dataset_clean.csv", parse_dates=["datetime"])
df = df.sort_values("datetime")

df.head()


,datetime,V,Np,Bz,Bt,kp
0,2018-01-01 00:00:00+00:00,393.000000,16.100000,-3.800000,9.666667,3.333
1,2018-01-01 03:00:00+00:00,417.666667,17.166667,-0.866667,10.066667,3.667
2,2018-01-01 06:00:00+00:00,437.000000,8.700000,-0.466667,6.900000,2.333
3,2018-01-01 09:00:00+00:00,433.000000,7.033333,-0.833333,6.966667,2.333
4,2018-01-01 12:00:00+00:00,443.333333,5.266667,-1.533333,6.566667,2.667


In [ ]:
lags = [1, 2, 3]

for lag in lags:
    df[f"Bz_lag_{lag}"] = df["Bz"].shift(lag)
    df[f"V_lag_{lag}"]  = df["V"].shift(lag)
    df[f"Np_lag_{lag}"] = df["Np"].shift(lag)


In [ ]:
windows = [2, 3]

for w in windows:
    df[f"Bz_mean_{w}"] = df["Bz"].rolling(w).mean()
    df[f"Bz_std_{w}"]  = df["Bz"].rolling(w).std()

    df[f"V_mean_{w}"]  = df["V"].rolling(w).mean()
    df[f"Np_mean_{w}"] = df["Np"].rolling(w).mean()


In [ ]:
df["Ey"] = df["V"] * df["Bz"].abs()


In [ ]:
df_fe = df.dropna().reset_index(drop=True)

df_fe.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19887 entries, 0 to 19886
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   datetime   19887 non-null  datetime64[ns, UTC]
 1   V          19887 non-null  float64            
 2   Np         19887 non-null  float64            
 3   Bz         19887 non-null  float64            
 4   Bt         19887 non-null  float64            
 5   kp         19887 non-null  float64            
 6   Bz_lag_1   19887 non-null  float64            
 7   V_lag_1    19887 non-null  float64            
 8   Np_lag_1   19887 non-null  float64            
 9   Bz_lag_2   19887 non-null  float64            
 10  V_lag_2    19887 non-null  float64            
 11  Np_lag_2   19887 non-null  float64            
 12  Bz_lag_3   19887 non-null  float64            
 13  V_lag_3    19887 non-null  float64            
 14  Np_lag_3   19887 non-null  float64            
 15  Bz

In [ ]:
df_fe.to_csv("geomagnetic_storm_ml_features.csv", index=False)



In [ ]:
from google.colab import files
files.download("geomagnetic_storm_ml_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

df = pd.read_csv("geomagnetic_storm_ml_features.csv", parse_dates=["datetime"])
df = df.sort_values("datetime").reset_index(drop=True)

df.head()


,datetime,V,Np,Bz,Bt,kp,Bz_lag_1,V_lag_1,Np_lag_1,Bz_lag_2,...,Np_lag_3,Bz_mean_2,Bz_std_2,V_mean_2,Np_mean_2,Bz_mean_3,Bz_std_3,V_mean_3,Np_mean_3,Ey
0,2018-01-01 09:00:00+00:00,433.000000,7.033333,-0.833333,6.966667,2.333,-0.466667,437.000000,8.700000,-0.866667,...,16.100000,-0.650000,0.259272,435.000000,7.866667,-0.722222,0.221944,429.222222,10.966667,360.833333
1,2018-01-01 12:00:00+00:00,443.333333,5.266667,-1.533333,6.566667,2.667,-0.833333,433.000000,7.033333,-0.466667,...,17.166667,-1.183333,0.494975,438.166667,6.150000,-0.944444,0.541944,437.777778,7.000000,679.777778
2,2018-01-01 15:00:00+00:00,470.333333,3.433333,0.200000,5.066667,1.000,-1.533333,443.333333,5.266667,-0.833333,...,8.700000,-0.666667,1.225652,456.833333,4.350000,-0.722222,0.871992,448.888889,5.244444,94.066667
3,2018-01-01 18:00:00+00:00,459.333333,3.900000,-0.033333,4.833333,1.000,0.200000,470.333333,3.433333,-1.533333,...,7.033333,0.083333,0.164992,464.833333,3.666667,-0.455556,0.940646,457.666667,4.200000,15.311111
4,2018-01-01 21:00:00+00:00,436.333333,4.333333,-1.066667,5.000000,1.333,-0.033333,459.333333,3.900000,0.200000,...,5.266667,-0.550000,0.730677,447.833333,4.116667,-0.300000,0.674125,455.333333,3.888889,465.422222


In [ ]:
y = df["kp"]
X = df.drop(columns=["datetime", "kp"])


In [ ]:
split_index = int(len(df) * 0.8)

X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    return {
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    }


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

rf_preds = rf_model.predict(X_test)

rf_metrics = evaluate_model(y_test, rf_preds)
rf_metrics


{'MAE': 0.5207743448478116,
 'RMSE': np.float64(0.6864996037664521),
 'R2': 0.7384572913292348}

In [ ]:
!pip install xgboost


In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror"
)

xgb_model.fit(X_train, y_train)

xgb_preds = xgb_model.predict(X_test)

xgb_metrics = evaluate_model(y_test, xgb_preds)
xgb_metrics


{'MAE': 0.5111653807780243,
 'RMSE': np.float64(0.6723568520815036),
 'R2': 0.749122504335235}

In [ ]:


results = pd.DataFrame.from_dict(
    {
        "Random Forest": rf_metrics,
        "XGBoost": xgb_metrics
    },
    orient="index"
)

results


,MAE,RMSE,R2
Random Forest,0.520774,0.686500,0.738457
XGBoost,0.511165,0.672357,0.749123
